## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [2]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download necessary NLTK resources if they haven't been downloaded before
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/roger.qiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roger.qiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

In [4]:
convention_db

In [5]:
convention_cur

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [6]:
# Query to list all tables in the SQLite database
tables_query = convention_cur.execute(
    "SELECT name FROM sqlite_master WHERE type='table';"
)

# Fetch all results
tables = tables_query.fetchall()

# Print the list of tables
print("Available tables in the database:", tables)


Available tables in the database: [('conventions',)]


In [7]:
query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions
                            ''')

query_results

In [8]:
# get the stopwords
stop_words = set(stopwords.words('english'))
list(stop_words)[:5]

['from', 'does', 'are', 'were', 'that']

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. As part of your cleaning process,
# remove the stopwords from the text. The second element of the sublist
# should be the party. 

for row in query_results:

    # join characters into 1 string, only get first row to exclude the party
    raw_text = ''.join(char for char in row[0])

    # loop through raw text and lowercase, do not include any puncuations
    cleaned_text = ''.join(char.lower() for char in raw_text if char not in string.punctuation)

    # convert the cleaned text into tokens
    tokens = word_tokenize(cleaned_text)
    
    # only keep tokens if they are not a stopword as filtered tokens
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # join tokens back into a single string with spaces
    final_text = ' '.join(filtered_tokens)
    
    # finally, append the cleaned data and the party to convention_data
    convention_data.append([final_text, row[1]])

In [11]:
convention_data[:1]

[['skip content company careers press freelancers blog × services transcription captions foreign subtitles translation freelancers contact login « return transcript library home transcript categories transcripts 2020 election transcripts classic speech transcripts congressional testimony hearing transcripts debate transcripts donald trump transcripts entertainment transcripts financial transcripts interview transcripts political transcripts press conference transcripts speech transcripts sports transcripts technology transcripts aug 21 2020 2020 democratic national convention dnc night 4 transcript rev › blog › transcripts › 2020 election transcripts › 2020 democratic national convention dnc night 4 transcript night 4 2020 democratic national convention dnc august 20 read full transcript event transcribe content try rev free save time transcribing captioning subtitling',
  'Democratic']]

Let's look at some random entries and see if they look right. 

In [12]:
random.choices(convention_data,k=5)

[['paycheck protection program mean company coronavirus struck', 'Republican'],
 ['joe bring us together end pandemic make sure prepared next one joe bring us together squarely face dismantle racial injustice furthering work generations joe believe build beloved community one strong decent kind one see ’ vision parents grandparents fought',
  'Democratic'],
 ['left ’ backward view see america free exceptional nation earth instead see wicked nation must punished sins opponents say redemption come giving power tired anthem spoken every repressive movement throughout history country ’ look career politicians salvation america ’ turn government restore souls put faith almighty god joe biden savior america ’ soul destroyer america ’ jobs given chance destroyer american greatness 47 years joe biden took donations blue collar workers gave hugs even kisses told felt pain flew back washington voted ship jobs china many distant lands joe biden spent entire career outsourcing dreams dreams americ

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [13]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2332 as features in the model.


In [30]:
list(feature_words)[:5]

['rnc', 'kind', 'remind', 'continues', 'resolve']

In [15]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """

    # Your code here

    # split the text into words
    words = text.split()

    # return the word and true as a dictionary, only if that word is a feature word
    ret_dict = {word: True for word in set(words) if word in fw}
    
    return(ret_dict)

In [16]:
# checks if feature words is not empty
assert(len(feature_words)>0)

# checks conv features function when the following input is given, returns certain words as keys
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})

assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [17]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

featuresets[:1]

[({'blog': True,
   'election': True,
   'skip': True,
   'trump': True,
   '×': True,
   'august': True,
   'full': True,
   'categories': True,
   'company': True,
   'subtitling': True,
   'political': True,
   'event': True,
   'captioning': True,
   'library': True,
   '2020': True,
   'debate': True,
   'aug': True,
   'contact': True,
   'time': True,
   'content': True,
   'transcribing': True,
   'translation': True,
   'transcripts': True,
   'testimony': True,
   'entertainment': True,
   'sports': True,
   'freelancers': True,
   'interview': True,
   'free': True,
   'foreign': True,
   'transcribe': True,
   'try': True,
   'hearing': True,
   'transcription': True,
   'classic': True,
   'democratic': True,
   'night': True,
   '›': True,
   'return': True,
   'save': True,
   'subtitles': True,
   'login': True,
   'national': True,
   'dnc': True,
   '20': True,
   'rev': True,
   'transcript': True,
   'convention': True,
   'read': True,
   'speech': True,
   'press'

In [18]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [19]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.496


In [20]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     27.1 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.8 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

There are a few interesting observations from this first naive bayes models and its most informative features. The model only has an accuracy of 50%, I'm curious to see why it performed so poorly and how future models will compare to it. As for the most important features, the results are very interesting. China appears to be the most important word for deciding parties, as well as votes and enforcement. Because China is a deciding feature for republicans, it must mean that the party brings it up much more than democrats. A big reason for this could be that the republicans in the convention have less favorable views of the US relationship and trade deals with China than comapred to democrats and therefore, bring it up much more. The most important democrat terms here appear to be votes and climate. I find it interesting that democrats bring up the term votes so much more than republicans, although the climate term makes sense. Finally, the vast majority of the important features are deciding for the republican party, my thoughts on this is that republicans in this convention tend to use the same key words much more frequently than democrats do and that democrats use a much larger variety of terms so their terms have less feature importance.

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [21]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [22]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [23]:
results[:1]

[('Mo Brooks',
  'Republican',
  b'"Brooks Joins Alabama Delegation in Voting Against Flawed Funding Bill" http://t.co/3CwjIWYsNq')]

In [24]:
tweet_data = []

for candidate, party, tweet in results:

    # decode to string format first, else run into error
    tweet = tweet.decode('utf-8')

    # join characters together
    raw_text = ''.join(char for char in tweet)
    
    # loop through raw text and lowercase, do not include any puncuations
    cleaned_text = ''.join(char.lower() for char in raw_text if char not in string.punctuation)
    
    # convert the cleaned text into tokens
    tokens = word_tokenize(cleaned_text)
    
    # only keep tokens if they are not a stopword as filtered tokens
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # join tokens back into a single string with spaces
    final_text = ' '.join(filtered_tokens)
    
    # append the cleaned, tokenized text and the candidate's party to tweet_data
    tweet_data.append([final_text, candidate, party])

tweet_data[:5]

[['brooks joins alabama delegation voting flawed funding bill httptco3cwjiwysnq',
  'Mo Brooks',
  'Republican'],
 ['brooks senate democrats allowing president give americans ’ jobs illegals securetheborder httpstcomzteax8xs6',
  'Mo Brooks',
  'Republican'],
 ['nasa square event sat 11am – 4pm stop amp hear incredible work done al05 downtownhsv httptcor9zy8wmepa',
  'Mo Brooks',
  'Republican'],
 ['trouble socialism eventually run peoples money margaret thatcher httpstcox97g7wzqwj',
  'Mo Brooks',
  'Republican'],
 ['trouble socialism eventually run peoples money – thatcher shell sorely missed httptcoz8gbndquh8',
  'Mo Brooks',
  'Republican']]

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [25]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

len(tweet_data_sample)

10

In [26]:

for tweet, candidate, party in tweet_data_sample :
    # Fill in the right-hand side above with code that estimates the actual party

    # convert tweets to features using the conv_features function and feature words defined earlier
    features = conv_features(tweet, feature_words)
    
    # use the model from earlier to classify the tweet
    estimated_party = classifier.classify(features)
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: ’ grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let ’ make even greater kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr cavs 

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [27]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0

results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 0, 'Democratic': 0}),
             'Democratic': defaultdict(int,
                         {'Republican': 0, 'Democratic': 0})})

In [28]:
num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, candidate, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    
    # convert tweets to features using the conv_features function and feature words defined earlier
    features = conv_features(tweet, feature_words)
    
    # use the model from earlier to classify the tweet
    estimated_party = classifier.classify(features)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [29]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3731, 'Democratic': 547}),
             'Democratic': defaultdict(int,
                         {'Republican': 4841, 'Democratic': 883})})

### Reflections

The classifier seems to have a bias toward labeling tweets as republican given that tweets from both parties are more frequently labeled as such. This could either be caused by an imbalance in the sample training data or a set of feature words are given more importance to to Republicans label regardless of the actual party.

For the results, of the total Republican tweets classified, 4291, about 86% are correctly labeled as Republican, and only 14% are misclassified as democrat, which is not bad. The issue, thought is the classification of democrat tweets, where out of the total 5711, only about 15% are correctly classified as Democratic, and a vast majority, 85% are misclassified as republican. Based on this, next steps could be to assess the training data, the models feature importance and make necesarry updates to improve the overall prediction accuracy.
